## some pratice of using pytorch

In [3]:
import torch as t
from torch.autograd import Variable

Pytorch 中的Autograd模块用于提供微分，帮助反向传播求导。autograd.Variable 是核心类，封装了Tensor. 并包含三个属性：
1. data
2. grad
3. grad_fn

In [5]:
x = Variable(t.ones(2,2), requires_grad = True)
print(x)

tensor([[1., 1.],
        [1., 1.]], requires_grad=True)


In [12]:
y = x.sum()
print(y)
y.grad_fn

tensor(4., grad_fn=<SumBackward0>)


In [13]:
y.backward()
x.grad

tensor([[4., 4.],
        [4., 4.]])

每次运行后，梯度都会累加到之前的梯度，故反向传播之前需要把他们清零

In [15]:
x.grad.data.zero_()
y.backward()
x.grad


tensor([[1., 1.],
        [1., 1.]])

torch.nn 是专门为神经网络设计的模块化接口，构建于Autograd之上。 nn.module 是重要的类

In [16]:
import torch.nn as nn
import torch.nn.functional as F

In [45]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x
    
net = Net()
print(net)
print(net.parameters)

Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)
<bound method Module.parameters of Net(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)>


In [43]:
for name,param in net.named_parameters():
    print(name, ': ', param.size())

conv1.weight :  torch.Size([6, 1, 5, 5])
conv1.bias :  torch.Size([6])
conv2.weight :  torch.Size([16, 6, 5, 5])
conv2.bias :  torch.Size([16])
fc1.weight :  torch.Size([120, 400])
fc1.bias :  torch.Size([120])
fc2.weight :  torch.Size([84, 120])
fc2.bias :  torch.Size([84])
fc3.weight :  torch.Size([10, 84])
fc3.bias :  torch.Size([10])


在输入时需要把Tensor 封装成Variable

In [55]:
input = Variable(t.randn(1, 1, 32, 32))
out = net(input)
print(out.size())
print(out)

net.zero_grad()  
out.backward(Variable(t.ones(1, 10)))
print('--'*40)
print(net.conv1.bias.grad)   #计算反向传播后conv1.bias的导数

torch.Size([1, 10])
tensor([[ 0.0782, -0.0188,  0.1065,  0.0996,  0.1524, -0.0084, -0.1102, -0.0115,
          0.0647,  0.0476]], grad_fn=<ThAddmmBackward>)
--------------------------------------------------------------------------------
tensor([ 0.0329,  0.0211, -0.0486,  0.0416, -0.0195,  0.0405])


### 损失函数  
nn.MSELoss 来计算均方误差， nn.CrossEntropyLoss 来计算交叉熵损失。

In [62]:
output = net(input)

target = Variable(t.rand(1,10))
criterion = nn.MSELoss()        #nn.MSELoss 是一个class,需先实例化
loss = criterion(output, target)
print(loss)

tensor(0.4176, grad_fn=<MseLossBackward>)


In [60]:
net.zero_grad()
loss.backward()
print(net.conv1.bias.grad)   #根据所选损失函数计算反向传播后conv1.bias的导数

tensor([-0.0011, -0.0002,  0.0037, -0.0020,  0.0036, -0.0005])


### 优化器
根据损失函数计算完梯度后，还需选择优化方法 来更新网络中的参数和权重。 如SGD：

weight = weight - learning_rate * gradient

In [ ]:
learning_rate = 0.01
for f in net.parameters():
    f.date.sub_(f.grad.data * learning_rate)      #手动实现SGD

torch.optim 中实现了深度学习中绝大多数的优化方法。

In [66]:
import torch.optim as optim

optimizer = optim.SGD(net.parameters(), lr = 0.01)  #lr为学习率

optimizer.zero_grad()

print(net.conv1.bias)

output = net(input)
loss = criterion(output, target)

loss.backward()

optimizer.step()  #更新参数
print('--'*40)
print(net.conv1.bias)

Parameter containing:
tensor([ 0.1853, -0.1959, -0.1373,  0.0577, -0.1046,  0.0327], requires_grad=True)
--------------------------------------------------------------------------------
Parameter containing:
tensor([ 0.1853, -0.1959, -0.1373,  0.0577, -0.1046,  0.0327], requires_grad=True)
